In [98]:
import tensorflow as tf
from models import decoder, encoder, vae
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pathlib
from constants import *

In [132]:
apes_info = pd.read_csv(APES_INFO_FILEPATH)
train_ids = apes_info.loc[apes_info["dataset"] == "train", "image"].to_list()
validation_ids = apes_info.loc[apes_info["dataset"] == "validation", "image"].to_list()
test_ids = apes_info.loc[apes_info["dataset"] == "test", "image"].to_list()
images_ids = sorted([item.stem for item in pathlib.Path(DATA_FILEPATH).iterdir() if item.suffix == ".png"])

In [133]:
dataset = (
    tf.keras.utils.image_dataset_from_directory(
        directory=DATA_FILEPATH,
        batch_size=1,
        image_size=IMAGE_SIZE,
        shuffle=False,
        labels=images_ids,
    )
    .unbatch()
    .map(lambda x, y: (x / 255, y))
)


@tf.autograph.experimental.do_not_convert
def select_x(x, _):
    return x

Found 10000 files belonging to 10000 classes.


In [134]:
decoder_model = decoder.build_decoder(LATENT_DIM)
encoder_model = encoder.build_encoder(LATENT_DIM)

vae_model = vae.VAE(encoder_model, decoder_model, 100, 1)
vae_model.load_weights("../data/models/vae/")

for i in range(len(vae_model.layers)):
    vae_model.layers[i].trainable = False

In [174]:
bck = pd.concat([pd.get_dummies(apes_info["Background"]), apes_info[["dataset"]]], axis=1)
bck_train = bck[bck["dataset"] == "train"].drop(columns="dataset")
bck_val = bck[bck["dataset"] == "validation"].drop(columns="dataset")

In [136]:
# inp = tf.keras.layers.Input((256, 256, 3))
# enc2 = tf.keras.Sequential(encoder_model.layers[1:-3])(inp)
# x = tf.keras.layers.Dense(32, activation='relu')(enc2)
# x = tf.keras.layers.Dense(16, activation='relu')(x)
# out1 = tf.keras.layers.Dense(8, activation='sigmoid')(x)
# out2 = tf.keras.layers.Dense(33, activation='sigmoid')(x)
#
# model = tf.keras.Model(inp, [out1, out2])

In [180]:
inp = tf.keras.layers.Input((256, 256, 3))
enc = encoder_model(inp)
concat = tf.keras.layers.Concatenate()([enc[0], enc[1]])
x = tf.keras.layers.Dense(64, activation="relu")(concat)
x = tf.keras.layers.Dense(64, activation="relu")(x)
x = tf.keras.layers.Dense(64, activation="relu")(x)
out = tf.keras.layers.Dense(8, activation="softmax")(x)
mod = tf.keras.Model(inp, out, name="abc")

In [181]:
mod.compile(optimizer="adam", loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), metrics=["accuracy"])

In [182]:
# ind1 = bck.value_counts().reset_index().reset_index().drop(columns="count")
# y1 = pd.get_dummies(pd.DataFrame(bck).merge(ind1)["index"])
#
# ind2 = mth.value_counts().reset_index().reset_index().drop(columns="count")
# y2 = pd.get_dummies(pd.DataFrame(mth).merge(ind2)["index"])
# y = pd.concat([y1, y2], axis=1)

In [176]:
y_ds_train = tf.data.Dataset.from_tensor_slices(bck_train)
x_ds_train = dataset.filter(lambda x, y: tf.math.reduce_any(y == train_ids)).map(select_x).batch(1)
ds_train = tf.data.Dataset.zip((x_ds_train.unbatch(), y_ds_train)).batch(32)

y_ds_val = tf.data.Dataset.from_tensor_slices(bck_val)
x_ds_val = dataset.filter(lambda x, y: tf.math.reduce_any(y == validation_ids)).map(select_x).batch(1)
ds_val = tf.data.Dataset.zip((x_ds_val.unbatch(), y_ds_val)).batch(3)

In [183]:
mod.fit(ds_train, validation_data=ds_val, epochs=5, batch_size=None)

Epoch 1/5


2023-05-19 16:30:33.481810: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [10000]
	 [[{{node Placeholder/_0}}]]
2023-05-19 16:30:33.482059: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype string and shape [10000]
	 [[{{node Placeholder/_4}}]]


219/219 [==============================] - 44s 198ms/step - loss: 0.1311 - accuracy: 0.8237 - val_loss: 0.0273 - val_accuracy: 1.0000
Epoch 2/5
219/219 [==============================] - 44s 201ms/step - loss: 0.0172 - accuracy: 0.9867 - val_loss: 0.0178 - val_accuracy: 1.0000
Epoch 3/5
219/219 [==============================] - ETA: 0s - loss: 0.0118 - accuracy: 0.9864

KeyboardInterrupt: 